In [ ]:
# This script is written to download my memories from FB
# For photos in album, FB provides an option to download directly
# However, to download the photos that you are tagged or you posted on timeline, you have to do it manually
# To avoid download the timeline photos 1-by-1 manually, I wrote this script to do it for me instead
# Please note that FB updates the app frequently so this script may not work when they changed the UI or class names

In [2]:
# Your Facebook credentials (keep these secret)
username = ""
password = ""

In [ ]:
# Get all photo urls from Facebook (this step is manual, you can automate it)
# 1. Go to https://www.facebook.com/[username]/photos (for photos that you are tagged)
# Or https://www.facebook.com/[username]/photos_by (for photos that you posted)
# 2. Inspect the webpage by right-clicking on a web page and selecting "Inspect" from the menu
# 3. Scroll to the end of the page (ensure all photos are loaded)
# 4. Once all photos are loaded, copy the <body> element from "Elements" tab and open it in your favourite IDE
# 5. Select all <a> tag occurences with class "x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g xt0b8zv x1lliihq x5yr21d x1n2onr6 xh8yej3", get the url in href attribute
# 6. Reformat all urls to be in array form and replace the urls variable below
urls = []

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys  # Import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os

counter = 1

image_urls = []

# Function to download an image
def download_image(image_url, destination_folder, image_name):
    response = requests.get(image_url)
    if response.status_code == 200:
        with open(os.path.join(destination_folder, image_name), 'wb') as file:
            file.write(response.content)

# Setup Selenium WebDriver
driver = webdriver.Chrome()  # Replace with your browser and WebDriver

# Directory to save images
destination_folder = "downloaded_images"
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Iterate over each URL
for url in urls:
    driver.get(url)

    # Login Facebook with your credentials for first URL
    if counter == 1:
        # Find the username and password fields
        username_field = driver.find_element(By.NAME, "email")
        password_field = driver.find_element(By.NAME, "pass")
        
        # Enter your credentials
        username_field.send_keys(username)
        password_field.send_keys(password)
        
        # Submit the form
        password_field.send_keys(Keys.RETURN)

        # If you have setup two factor authentication, please approve the access at this step
    
    # Wait for content to load
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "img.x85a59c.x193iq5w.x4fas0m.x19kjcj4")))

    # Find images with specific class names
    image = driver.find_element(By.CSS_SELECTOR, "img.x85a59c.x193iq5w.x4fas0m.x19kjcj4")
    src = image.get_attribute("src")
    image_urls.append(src)
    
    if src:
        download_image(src, destination_folder, f"image_{counter}.jpg")
        counter += 1
    else: 
        print("Fail: ", url)

# Close the browser
driver.quit()
